In [462]:
import edgar
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bsoup
import numpy as np

In [463]:
res = requests.get('https://etfdb.com/history-of-the-s-and-p-500/#2009')
soup = bsoup(res.content,'html.parser')

In [464]:
def extractTable(soup):
    '''Extract the top ten companies from 1980-2013 '''
    table = []
    for div in soup.find_all('div', {'id':'sections'}):
        for a in div.find_all('div', {'class':'dataRow'}):
            for s in a.find_all('span', {'class': 'rowTitle'}):
                table+=s
    return(table)

In [465]:
tbl = extractTable(soup)
# Remove companies 1980-1999
Lst = tbl[200:]

def getComp(Lst):
    return [Lst[i::10] for i in range(10)]
# Yield successive n-sized 
def divide_chunks(Lst, n): 

    for i in range(0, len(Lst), n):  
        yield Lst[i:i + n] 
        


In [466]:
# Split into chunks of 10 CO per year 
tbl = list(divide_chunks(Lst, 10))
colNames=[i for i in range(2000,2014)]

In [467]:
# Make df
dfTop10 = pd.DataFrame(tbl).transpose()
dfTop10.columns = colNames

In [468]:
# Remove punctuation and uppercase 
for i in colNames:
    dfTop10[i] = dfTop10[i].str.upper().str.strip(string.punctuation).str.strip('\n').str.strip()


In [469]:
dfTop10

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,GENERAL ELECTRIC,GENERAL ELECTRIC,MICROSOFT,GENERAL ELECTRIC,GENERAL ELECTRIC,GENERAL ELECTRIC,EXXON MOBIL,EXXON MOBIL,EXXON MOBIL,EXXON MOBIL,EXXON MOBIL,EXXON MOBIL,APPLE,APPLE
1,EXXON MOBIL,MICROSOFT,GENERAL ELECTRIC,MICROSOFT,EXXON MOBIL,EXXON MOBIL,GENERAL ELECTRIC,GENERAL ELECTRIC,WAL-MART STORES,MICROSOFT,APPLE,APPLE,EXXON MOBIL,EXXON MOBIL
2,PFIZER,EXXON MOBIL,EXXON MOBIL,EXXON MOBIL,MICROSOFT,MICROSOFT,MICROSOFT,MICROSOFT,PROCTER & GAMBLE,WAL-MART STORES,MICROSOFT,MICROSOFT,MICROSOFT,GOOGLE
3,CITIGROUP,CITIGROUP,WAL-MART STORES,PFIZER,CITIGROUP,CITIGROUP,CITIGROUP,AT&T,MICROSOFT,GOOGLE,BERKSHIRE HATHAWAY,IBM,GENERAL ELECTRIC,MICROSOFT
4,CISCO SYSTEMS,WAL-MART STORES,PFIZER,CITIGROUP,WAL-MART STORES,PROCTER & GAMBLE,BANK OF AMERICA,PROCTER & GAMBLE,AT&T,APPLE,GENERAL ELECTRIC,CHEVRON,IBM,GENERAL ELECTRIC
5,WAL-MART STORES,PFIZER,CITIGROUP,WAL-MART STORES,PFIZER,WAL-MART STORES,PROCTER & GAMBLE,GOOGLE,JOHNSON & JOHNSON,JOHNSON & JOHNSON,WAL-MART STORES,GOOGLE,CHEVRON,JOHNSON & JOHNSON
6,MICROSOFT,INTEL,JOHNSON & JOHNSON,INTEL,BANK OF AMERICA,BANK OF AMERICA,WAL-MART STORES,CHEVRON,GENERAL ELECTRIC,PROCTER & GAMBLE,GOOGLE,WAL-MART STORES,JOHNSON & JOHNSON,CHEVRON
7,AIG,IBM,AIG,AIG,JOHNSON & JOHNSON,JOHNSON & JOHNSON,JOHNSON & JOHNSON,JOHNSON & JOHNSON,CHEVRON CORP,IBM,CHEVRON CORP,GENERAL ELECTRIC,AT&T,PROCTER & GAMBLE
8,MERCK,AIG,IBM,CISCO SYSTEMS,AIG,AIG,PFIZER,WAL-MART STORES,PFIZER,AT&T,IBM,BERKSHIRE HATHAWAY,PROCTER & GAMBLE,JPMORGAN CHASE
9,INTEL,JOHNSON & JOHNSON,MERCK,IBM,IBM,PFIZER,AIG,BANK OF AMERICA,JPMORGAN CHASE,JPMORGAN CHASE,PROCTER & GAMBLE,PROCTER & GAMBLE,PFIZER,WELLS FARGO


In [470]:
dfTop10.replace({'CHEVRON CORP': 'CHEVRON', 'CHEVRON': 'CHEVRON CORP','MICROSOFT': 'MICROSOFT CORP','GENERAL ELECTRIC': 'GENERAL ELECTRIC CO', 'EXXON MOBIL':'EXXON MOBIL CORP', 'PFIZER':'PFIZER INC', 'CITIGROUP':'CITIGROUP INC', 'CISCO SYSTEMS':'CISCO SYSTEMS INC', 'WAL-MART STORES' :'WALMART INC','AIG':'AMERICAN INTERNATIONAL GROUP','MERCK':'MERCK & CO','INTEL':'INTEL CORP','PROCTER & GAMBLE':'PROCTER & GAMBLE CO','AT&T':'AT&T CORP','JPMORGAN CHASE':'JPMORGAN CHASE & CO','GOOGLE':'ALPHABET INC','APPLE':'APPLE INC','BANK OF AMERICA':'BANK OF AMERICA CORP','IBM':'INTL BUSINESS MACHINES CORP','CHEVRON':'CHEVRON CORP','WELLS FARGO':'WELLS FARGO & CO'}, regex=True, inplace=True)

In [471]:
dfCik = pd.read_csv('2000_2013_s&p.csv')
dfCik.drop(['gvkey','gvkeyx','from','thru','tic'], axis=1, inplace=True)
dfCik['co_cik'] = dfCik['co_cik'].fillna(0).astype(int).apply(lambda x: '{0:0>7}'.format(x))
cikDic = pd.Series(dfCik.co_conm.values,index=dfCik.co_cik).to_dict()
cikDic = dict((v,k) for k,v in cikDic.items())

In [472]:
from collections import defaultdict
dic = defaultdict(None)
for yr in colNames:  
    for comp in dfTop10[yr]:
        if comp in cikDic:
            dic[comp] = cikDic[comp]
        else:
            dic[comp] = None
        

In [475]:
for yr in colNames:  
    for comp in dfTop10[yr]:
        dfTop10[yr][comp] = (comp, cikDic[comp])

                           2000                          2001  \
0           GENERAL ELECTRIC CO           GENERAL ELECTRIC CO   
1              EXXON MOBIL CORP                MICROSOFT CORP   
2                    PFIZER INC              EXXON MOBIL CORP   
3                 CITIGROUP INC                 CITIGROUP INC   
4             CISCO SYSTEMS INC                   WALMART INC   
5                   WALMART INC                    PFIZER INC   
6                MICROSOFT CORP                    INTEL CORP   
7  AMERICAN INTERNATIONAL GROUP   INTL BUSINESS MACHINES CORP   
8                    MERCK & CO  AMERICAN INTERNATIONAL GROUP   
9                    INTEL CORP             JOHNSON & JOHNSON   

                           2002                          2003  \
0                MICROSOFT CORP           GENERAL ELECTRIC CO   
1           GENERAL ELECTRIC CO                MICROSOFT CORP   
2              EXXON MOBIL CORP              EXXON MOBIL CORP   
3                   WALM

KeyError: ('GENERAL ELECTRIC CO', '0040545')

In [474]:
dfTop10

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,GENERAL ELECTRIC CO,GENERAL ELECTRIC CO,MICROSOFT CORP,GENERAL ELECTRIC CO,GENERAL ELECTRIC CO,GENERAL ELECTRIC CO,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,APPLE INC,APPLE INC
1,EXXON MOBIL CORP,MICROSOFT CORP,GENERAL ELECTRIC CO,MICROSOFT CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,GENERAL ELECTRIC CO,GENERAL ELECTRIC CO,WALMART INC,MICROSOFT CORP,APPLE INC,APPLE INC,EXXON MOBIL CORP,EXXON MOBIL CORP
2,PFIZER INC,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,MICROSOFT CORP,MICROSOFT CORP,MICROSOFT CORP,MICROSOFT CORP,PROCTER & GAMBLE CO,WALMART INC,MICROSOFT CORP,MICROSOFT CORP,MICROSOFT CORP,ALPHABET INC
3,CITIGROUP INC,CITIGROUP INC,WALMART INC,PFIZER INC,CITIGROUP INC,CITIGROUP INC,CITIGROUP INC,AT&T CORP,MICROSOFT CORP,ALPHABET INC,BERKSHIRE HATHAWAY,INTL BUSINESS MACHINES CORP,GENERAL ELECTRIC CO,MICROSOFT CORP
4,CISCO SYSTEMS INC,WALMART INC,PFIZER INC,CITIGROUP INC,WALMART INC,PROCTER & GAMBLE CO,BANK OF AMERICA CORP,PROCTER & GAMBLE CO,AT&T CORP,APPLE INC,GENERAL ELECTRIC CO,CHEVRON CORP,INTL BUSINESS MACHINES CORP,GENERAL ELECTRIC CO
5,WALMART INC,PFIZER INC,CITIGROUP INC,WALMART INC,PFIZER INC,WALMART INC,PROCTER & GAMBLE CO,ALPHABET INC,JOHNSON & JOHNSON,JOHNSON & JOHNSON,WALMART INC,ALPHABET INC,CHEVRON CORP,JOHNSON & JOHNSON
6,MICROSOFT CORP,INTEL CORP,JOHNSON & JOHNSON,INTEL CORP,BANK OF AMERICA CORP,BANK OF AMERICA CORP,WALMART INC,CHEVRON CORP,GENERAL ELECTRIC CO,PROCTER & GAMBLE CO,ALPHABET INC,WALMART INC,JOHNSON & JOHNSON,CHEVRON CORP
7,AMERICAN INTERNATIONAL GROUP,INTL BUSINESS MACHINES CORP,AMERICAN INTERNATIONAL GROUP,AMERICAN INTERNATIONAL GROUP,JOHNSON & JOHNSON,JOHNSON & JOHNSON,JOHNSON & JOHNSON,JOHNSON & JOHNSON,CHEVRON CORP,INTL BUSINESS MACHINES CORP,CHEVRON CORP,GENERAL ELECTRIC CO,AT&T CORP,PROCTER & GAMBLE CO
8,MERCK & CO,AMERICAN INTERNATIONAL GROUP,INTL BUSINESS MACHINES CORP,CISCO SYSTEMS INC,AMERICAN INTERNATIONAL GROUP,AMERICAN INTERNATIONAL GROUP,PFIZER INC,WALMART INC,PFIZER INC,AT&T CORP,INTL BUSINESS MACHINES CORP,BERKSHIRE HATHAWAY,PROCTER & GAMBLE CO,JPMORGAN CHASE & CO
9,INTEL CORP,JOHNSON & JOHNSON,MERCK & CO,INTL BUSINESS MACHINES CORP,INTL BUSINESS MACHINES CORP,PFIZER INC,AMERICAN INTERNATIONAL GROUP,BANK OF AMERICA CORP,JPMORGAN CHASE & CO,JPMORGAN CHASE & CO,PROCTER & GAMBLE CO,PROCTER & GAMBLE CO,PFIZER INC,WELLS FARGO & CO
